In [2]:
import pandas as pd
from read_db.CH import Getch
import pandahouse
from datetime import date,timedelta, datetime
import telegram
import seaborn as sns
import matplotlib.pyplot as plt
import io
import numpy as np
from datetime import datetime, date, time

In [23]:
from airflow import DAG
from airflow.operators.python import PythonOperator # Так как мы пишет такси в питоне
from datetime import datetime

In [46]:
#TASK 1
feed = Getch(
        '''
        select user_id,source ,gender, age, os
                    ,multiIf(age = 0, 'no info',
                 age <=20 , '1-20',
                 age <=30 , '21-30',
                 age <=40 , '31-40',
                 age <=50 , '41-50',
                 '50+') as age_type
                ,toDate(time) as yesterday
                ,countIf(post_id,action ='like') as likes
                ,countIf(post_id,action='view') as views
        from simulator_20220520.feed_actions
        where toDate(time) = today()-1
        group by user_id, source, gender, age, os, toDate(time)
         ''').df


In [47]:
#TASK 2
message = Getch(
'''
with s as (select user_id, os,age,gender
                 , toDate(time) as yesterday
                 , count(reciever_id) as messages_sent
                 , count(distinct reciever_id) as users_sent
            from simulator_20220520.message_actions 
            where toDate(time) =today()-1
            group by user_id, os,age,gender,yesterday)
, r as (select reciever_id
                 , toDate(time) as yesterday
                 , count(user_id) as messages_received
                 , count(distinct user_id) as users_received
            from simulator_20220520.message_actions 
            where toDate(time) =today()-1
            group by reciever_id,yesterday)
            
            
select if(user_id = 0,reciever_id, user_id) as user_id
,if(os='','no info',os) as os
,age
,multiIf(age = 0, 'no info',
     age <=20 , '1-20',
     age <=30 , '21-30',
     age <=40 , '31-40',
     age <=50 , '41-50',
     '50+') as age_type
,if(age=0,2,gender) as gender
,if(s.yesterday ='1970-01-01',r.yesterday,s.yesterday) as yesterday
,messages_sent
,users_sent
,messages_received
,users_received
from s
full join r on r.reciever_id = s.user_id
'''
).df

In [48]:
message.head(2)

,user_id,os,age,age_type,gender,yesterday,messages_sent,users_sent,messages_received,users_received
0,110797,iOS,41,41-50,1,2022-06-09,5,2,19,8
1,17506,iOS,29,21-30,0,2022-06-09,8,8,3,3


In [49]:
#TASK 3
df = feed.merge(message, left_on = ['user_id','os','age','age_type','gender','yesterday'], right_on =['user_id','os','age','age_type','gender','yesterday'], how ='outer').fillna(0)
df.head(2)

,user_id,source,gender,age,os,age_type,yesterday,likes,views,messages_sent,users_sent,messages_received,users_received
0,11054,ads,1,30,Android,21-30,2022-06-09,16.0,82.0,0.0,0.0,0.0,0.0
1,81301,ads,1,44,Android,41-50,2022-06-09,4.0,17.0,0.0,0.0,0.0,0.0


In [50]:
df[['likes','views','messages_sent','users_sent','messages_received','users_received']]

,likes,views,messages_sent,users_sent,messages_received,users_received
0,16.0,82.0,0.0,0.0,0.0,0.0
1,4.0,17.0,0.0,0.0,0.0,0.0
2,3.0,17.0,0.0,0.0,0.0,0.0
3,19.0,89.0,0.0,0.0,0.0,0.0
4,12.0,82.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
20405,0.0,0.0,0.0,0.0,1.0,1.0
20406,0.0,0.0,0.0,0.0,3.0,3.0
20407,0.0,0.0,0.0,0.0,7.0,7.0
20408,0.0,0.0,0.0,0.0,10.0,10.0


In [51]:
#TASK 4 - пол
df_gender = df[['gender','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','gender'], as_index=False).agg('sum')
df_gender['metric'] ='gender'
df_gender = df_gender.rename(columns = {'gender':'metric_value'})
df_gender['metric_value']= df_gender['metric_value'].apply(lambda x: 'no info' if x == 2 else x)
df_gender

,yesterday,metric_value,likes,views,messages_sent,users_sent,messages_received,users_received,metric
0,2022-06-09,0,71220.0,339587.0,5529.0,4162.0,5249.0,4124.0,gender
1,2022-06-09,1,89720.0,430939.0,7372.0,5595.0,7616.0,5597.0,gender
2,2022-06-09,no info,0.0,0.0,0.0,0.0,36.0,36.0,gender


In [52]:
#TASK 5 - возраст
df_age = df[['age_type','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','age_type'], as_index=False).agg('sum')
df_age['metric'] ='age_type'
df_age = df_age.rename(columns = {'age_type':'metric_value'})
df_age

,yesterday,metric_value,likes,views,messages_sent,users_sent,messages_received,users_received,metric
0,2022-06-09,1-20,51007.0,244993.0,4020.0,3538.0,4078.0,3111.0,age_type
1,2022-06-09,21-30,61408.0,293452.0,4688.0,4519.0,5660.0,4938.0,age_type
2,2022-06-09,31-40,29454.0,140263.0,2703.0,1236.0,2409.0,1455.0,age_type
3,2022-06-09,41-50,13806.0,66720.0,1062.0,325.0,232.0,114.0,age_type
4,2022-06-09,50+,5265.0,25098.0,428.0,139.0,486.0,103.0,age_type
5,2022-06-09,no info,0.0,0.0,0.0,0.0,36.0,36.0,age_type


In [53]:
#TASK 6 - os
df_os = df[['os','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','os'], as_index=False).agg('sum')
df_os['metric'] ='os'
df_os = df_os.rename(columns = {'os':'metric_value'})
df_os

,yesterday,metric_value,likes,views,messages_sent,users_sent,messages_received,users_received,metric
0,2022-06-09,Android,104538.0,501945.0,8358.0,6201.0,8235.0,6070.0,os
1,2022-06-09,iOS,56402.0,268581.0,4543.0,3556.0,4630.0,3651.0,os
2,2022-06-09,no info,0.0,0.0,0.0,0.0,36.0,36.0,os


In [54]:
#TASK 7
df_final = pd.concat([df_gender,df_os,df_age])
df_final = df_final[['yesterday','metric','metric_value','likes','views','messages_sent','users_sent','messages_received','users_received']].rename(columns={'yesterday':'eventdate'})
df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']] = df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']].astype(int)
#df_final['eventdate'] =date(df_final['eventdate'])
df_final.dtypes

eventdate            datetime64[ns]
metric                       object
metric_value                 object
likes                         int64
views                         int64
messages_sent                 int64
users_sent                    int64
messages_received             int64
users_received                int64
dtype: object

In [9]:
#подключение к новой базе
connection_ch = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': '656e2b0c9c',
    'user': 'student-rw',
    'database': 'test'}

q1 = '''
CREATE TABLE IF NOT EXISTS test.eorlova_2
(date Date
, metric String
, metric_value String
, likes UInt64
, views UInt64
, messages_sent UInt64
, users_sent UInt64
, messages_received UInt64
, users_received UInt64) 
ENGINE = MergeTree()
--ORDER BY date
'''
pandahouse.execute(query=q1, connection=connection_ch)

b''

In [15]:
connection_ch = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': '656e2b0c9c',
    'user': 'student-rw',
    'database': 'test'}
q = '''
CREATE TABLE IF NOT EXISTS test.eorlova_3 (
date Date,
variable String,
value String, 
messages_sent UInt64

)
ENGINE = Log()
'''
pandahouse.execute(query=q, connection=connection_ch)

b''

In [58]:
max_date

eventdate   2022-06-07
Name: 0, dtype: datetime64[ns]

In [57]:
q2 = 'SELECT max(eventdate) as eventdate FROM test.eorlova'
max_date = pandahouse.read_clickhouse(q2, connection=connection_ch)

max_date = max_date.iloc[0]


if (max_date == df_final.eventdate.max())[0]:
    q3 = '''
    ALTER TABLE test.eorlova DELETE WHERE eventdate == today()-1
    '''
    pandahouse.execute(query=q3, connection=connection_ch)
    pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
else:
    pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)

In [59]:
import pandas as pd
from CH import Getch
import pandahouse
from datetime import date,timedelta, datetime
import telegram
import io
import numpy as np

from airflow import DAG
from airflow.operators.python_operator import PythonOperator # Так как мы пишет такси в питоне

#Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'e-orlova-7',
    'depends_on_past' : False,
    'retries':2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022,6,10),
}

#Интеравал запуска Даг
schedule_interval = '0 12 * * *'
dag = DAG('DAG_orlova', default_args=default_args, schedule_interval=schedule_interval)

def foo1():
    feed = Getch(
        '''
        select user_id,source ,gender, age, os
                    ,multiIf(age = 0, 'no info',
                 age <=20 , '1-20',
                 age <=30 , '21-30',
                 age <=40 , '31-40',
                 age <=50 , '41-50',
                 '50+') as age_type
                ,toDate(time) as yesterday
                ,countIf(post_id,action ='like') as likes
                ,countIf(post_id,action='view') as views
        from simulator_20220520.feed_actions
        where toDate(time) = today()-1
        group by user_id, source, gender, age, os, toDate(time)
        ''').df
    return feed

def foo2():
    #TASK 2
    message = Getch(
    '''
    with s as (select user_id, os,age,gender
                     , toDate(time) as yesterday
                     , count(reciever_id) as messages_sent
                     , count(distinct reciever_id) as users_sent
                from simulator_20220520.message_actions 
                where toDate(time) =today()-1
                group by user_id, os,age,gender,yesterday)
    , r as (select reciever_id
                     , toDate(time) as yesterday
                     , count(user_id) as messages_received
                     , count(distinct user_id) as users_received
                from simulator_20220520.message_actions 
                where toDate(time) =today()-1
                group by reciever_id,yesterday)


    select if(user_id = 0,reciever_id, user_id) as user_id
    ,if(os='','no info',os) as os
    ,age
    ,multiIf(age = 0, 'no info',
         age <=20 , '1-20',
         age <=30 , '21-30',
         age <=40 , '31-40',
         age <=50 , '41-50',
         '50+') as age_type
    ,if(age=0,2,gender) as gender
    ,if(s.yesterday ='1970-01-01',r.yesterday,s.yesterday) as yesterday
    ,messages_sent
    ,users_sent
    ,messages_received
    ,users_received
    from s
    full join r on r.reciever_id = s.user_id
    '''
    ).df
    return message

def foo3():
    df = feed.merge(message, left_on = ['user_id','os','age','age_type','gender','yesterday'], right_on =['user_id','os','age','age_type','gender','yesterday'], how ='outer').fillna(0)
    return df

def foo4():
    #TASK 4 - пол
    df_gender = df[['gender','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','gender'], as_index=False).agg('sum')
    df_gender['metric'] ='gender'
    df_gender = df_gender.rename(columns = {'gender':'metric_value'})
    df_gender['metric_value']= df_gender['metric_value'].apply(lambda x: 'no info' if x == 2 else x)
    return df_gender
    
def foo5():
    #TASK 5 - возраст
    df_age = df[['age_type','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','age_type'], as_index=False).agg('sum')
    df_age['metric'] ='age_type'
    df_age = df_age.rename(columns = {'age_type':'metric_value'})
    return df_age

def foo6():
    #TASK 6 - os
    df_os = df[['os','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','os'], as_index=False).agg('sum')
    df_os['metric'] ='os'
    df_os = df_os.rename(columns = {'os':'metric_value'})
    return df_os

def foo7():
    #TASK 7
    df_final = pd.concat([df_gender,df_os,df_age])
    df_final = df_final[['yesterday','metric','metric_value','likes','views','messages_sent','users_sent','messages_received','users_received']].rename(columns={'yesterday':'eventdate'})
    df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']] = df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']].astype(int)
    
@task
def into_base(df_final):
    #подключение к новой базе
    connection_ch = {
        'host': 'https://clickhouse.lab.karpov.courses',
        'password': '656e2b0c9c',
        'user': 'student-rw',
        'database': 'test'}

    q1 = '''
    CREATE TABLE IF NOT EXISTS test.eorlova
    (eventdate Date
    , metric String
    , metric_value String
    , likes UInt64
    , views UInt64
    , messages_sent UInt64
    , users_sent UInt64
    , messages_received UInt64
    , users_received UInt64) 
    ENGINE = MergeTree()
    ORDER BY eventdate
    '''
    pandahouse.execute(query=q1, connection=connection_ch)
    
    q2 = 'SELECT max(eventdate) as eventdate FROM test.eorlova'
    max_date = pandahouse.read_clickhouse(q2, connection=connection)
    max_date = max_date.iloc[0]

    if (max_date == df_final.eventdate.max())[0]:
        q3 = '''
        ALTER TABLE test.eorlova DELETE WHERE eventdate == today()-1
        '''
        pandahouse.execute(query=q3, connection=connection_ch)
        pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
    else:
        pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
     
    return 
 

t1 = PythonOperator(task_id='foo1', # Название таска
                    python_callable=foo1, # Название функции
                    dag=dag) # Параметры DAG

t2 = PythonOperator(task_id='foo2', # Название таска
                    python_callable=foo2, # Название функции
                    dag=dag) # Параметры DAG

t3 = PythonOperator(task_id='foo3', # Название таска
                    python_callable=foo3, # Название функции
                    dag=dag) # Параметры DAG

t4 = PythonOperator(task_id='foo4', # Название таска
                    python_callable=foo4, # Название функции
                    dag=dag) # Параметры DAG

t5 = PythonOperator(task_id='foo5', # Название таска
                    python_callable=foo5, # Название функции
                    dag=dag) # Параметры DAG

t6 = PythonOperator(task_id='foo6', # Название таска
                    python_callable=foo6, # Название функции
                    dag=dag) # Параметры DAG

t7 = PythonOperator(task_id='foo7', # Название таска
                    python_callable=foo7, # Название функции
                    dag=dag) # Параметры DAG


[t1,t2] >> t3 >> [t4,t5,t6] >> t7




NameError: name 'task' is not defined

In [2]:
import pandas as pd
from CH import Getch
import pandahouse
from datetime import date, timedelta, datetime
import telegram
import io
import numpy as np

from airflow import DAG
from airflow.operators.python_operator import PythonOperator  # Так как мы пишет такси в питоне

# Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'e-orlova-7',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 6, 2),
}

# Интеравал запуска Даг
schedule_interval = '0 12 * * *'
dag = DAG('dag_orlova', default_args=default_args, schedule_interval=schedule_interval)

#@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def dag_orlova():
#    @task
    def foo1():
        feed = Getch(
            '''
            select user_id,source ,gender, age, os
                        ,multiIf(age = 0, 'no info',
                     age <=20 , '1-20',
                     age <=30 , '21-30',
                     age <=40 , '31-40',
                     age <=50 , '41-50',
                     '50+') as age_type
                    ,toDate(time) as yesterday
                    ,countIf(post_id,action ='like') as likes
                    ,countIf(post_id,action='view') as views
            from simulator_20220520.feed_actions
            where toDate(time) = today()-1
            group by user_id, source, gender, age, os, toDate(time)
            ''').df
        return feed

    def foo2():
        #TASK 2
        message = Getch(
        '''
        with s as (select user_id, os,age,gender
                         , toDate(time) as yesterday
                         , count(reciever_id) as messages_sent
                         , count(distinct reciever_id) as users_sent
                    from simulator_20220520.message_actions 
                    where toDate(time) =today()-1
                    group by user_id, os,age,gender,yesterday)
        , r as (select reciever_id
                         , toDate(time) as yesterday
                         , count(user_id) as messages_received
                         , count(distinct user_id) as users_received
                    from simulator_20220520.message_actions 
                    where toDate(time) =today()-1
                    group by reciever_id,yesterday)


        select if(user_id = 0,reciever_id, user_id) as user_id
        ,if(os='','no info',os) as os
        ,age
        ,multiIf(age = 0, 'no info',
             age <=20 , '1-20',
             age <=30 , '21-30',
             age <=40 , '31-40',
             age <=50 , '41-50',
             '50+') as age_type
        ,if(age=0,2,gender) as gender
        ,if(s.yesterday ='1970-01-01',r.yesterday,s.yesterday) as yesterday
        ,messages_sent
        ,users_sent
        ,messages_received
        ,users_received
        from s
        full join r on r.reciever_id = s.user_id
        '''
        ).df
        return message

    def foo3(feed,message):
        df = feed.merge(message, left_on = ['user_id','os','age','age_type','gender','yesterday'], right_on =['user_id','os','age','age_type','gender','yesterday'], how ='outer').fillna(0)
        return df

    def foo4(df):
        #TASK 4 - пол
        df_gender = df[['gender','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','gender'], as_index=False).agg('sum')
        df_gender['metric'] ='gender'
        df_gender = df_gender.rename(columns = {'gender':'metric_value'})
        df_gender['metric_value']= df_gender['metric_value'].apply(lambda x: 'no info' if x == 2 else x)
        return df_gender

    def foo5(df):
        #TASK 5 - возраст
        df_age = df[['age_type','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','age_type'], as_index=False).agg('sum')
        df_age['metric'] ='age_type'
        df_age = df_age.rename(columns = {'age_type':'metric_value'})
        return df_age

    def foo6(df):
        #TASK 6 - os
        df_os = df[['os','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','os'], as_index=False).agg('sum')
        df_os['metric'] ='os'
        df_os = df_os.rename(columns = {'os':'metric_value'})
        return df_os

    def foo7(df_gender,df_os,df_age):
        #TASK 7
        df_final = pd.concat([df_gender,df_os,df_age])
        df_final = df_final[['yesterday','metric','metric_value','likes','views','messages_sent','users_sent','messages_received','users_received']].rename(columns={'yesterday':'eventdate'})
        df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']] = df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']].astype(int)
        return df_final

    def into_base(df_final):
        #подключение к новой базе
        connection_ch = {
            'host': 'https://clickhouse.lab.karpov.courses',
            'password': '656e2b0c9c',
            'user': 'student-rw',
            'database': 'test'}

        q1 = '''
        CREATE TABLE IF NOT EXISTS test.eorlova
        (eventdate Date
        , metric String
        , metric_value String
        , likes UInt64
        , views UInt64
        , messages_sent UInt64f
        , users_sent UInt64
        , messages_received UInt64
        , users_received UInt64) 
        ENGINE = MergeTree()
        ORDER BY eventdate
        '''
        pandahouse.execute(query=q1, connection=connection_ch)

        q2 = 'SELECT max(eventdate) as eventdate FROM test.eorlova'
        max_date = pandahouse.read_clickhouse(q2, connection=connection_ch)
        max_date = max_date.iloc[0]

        if (max_date == df_final.eventdate.max())[0]:
            q3 = '''
            ALTER TABLE test.eorlova DELETE WHERE eventdate == today()-1
            '''
            pandahouse.execute(query=q3, connection=connection_ch)
            pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
        else:
            pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)

        return 
    print('ok')

    #with DAG(
    #    'dag_orlova',default_args=default_args, schedule_interval=schedule_interval, catchup=False) as dag:

t1 = PythonOperator(task_id='foo1',  # Название таска
                            python_callable=foo1,  # Название функции
                            dag=dag)  # Параметры DAG

t2 = PythonOperator(task_id='foo2',  # Название таска
                            python_callable=foo2,  # Название функции
                            dag=dag)  # Параметры DAG

t3 = PythonOperator(task_id='foo3',  # Название таска
                            python_callable=foo3,  # Название функции
                            dag=dag)  # Параметры DAG

t4 = PythonOperator(task_id='foo4',  # Название таска
                            python_callable=foo4,  # Название функции
                            dag=dag)  # Параметры DAG

t5 = PythonOperator(task_id='foo5',  # Название таска
                            python_callable=foo5,  # Название функции
                            dag=dag)  # Параметры DAG

t6 = PythonOperator(task_id='foo6',  # Название таска
                            python_callable=foo6,  # Название функции
                            dag=dag)  # Параметры DAG

t7 = PythonOperator(task_id='foo7',  # Название таска
                            python_callable=foo7,  # Название функции
                            dag=dag)  # Параметры DAG

t8 = PythonOperator(task_id='into_base',  # Название таска
                            python_callable=into_base,  # Название функции
                            dag=dag)  # Параметры DAG

[t1, t2] >> t3 >> [t4, t5, t6] >> t7 >> t8

dag_orlova = dag_orlova()


/tmp/ipykernel_45/4056895360.py:10 DeprecationWarning: This module is deprecated. Please use `airflow.operators.python`.

NameError: name 'foo1' is not defined

In [77]:
feed = foo1()
message = foo2()
df = foo3(feed, message)
df_g = foo4(df)
df_a = foo5(df)
df_o = foo6(df)
df_final = foo7(df_g, df_a, df_o)
into_base(df_final)

In [76]:
def foo1():
    feed = Getch(
        '''
        select user_id,source ,gender, age, os
                    ,multiIf(age = 0, 'no info',
                 age <=20 , '1-20',
                 age <=30 , '21-30',
                 age <=40 , '31-40',
                 age <=50 , '41-50',
                 '50+') as age_type
                ,toDate(time) as yesterday
                ,countIf(post_id,action ='like') as likes
                ,countIf(post_id,action='view') as views
        from simulator_20220520.feed_actions
        where toDate(time) = today()-1
        group by user_id, source, gender, age, os, toDate(time)
        ''').df
    return feed

def foo2():
    #TASK 2
    message = Getch(
    '''
    with s as (select user_id, os,age,gender
                     , toDate(time) as yesterday
                     , count(reciever_id) as messages_sent
                     , count(distinct reciever_id) as users_sent
                from simulator_20220520.message_actions 
                where toDate(time) =today()-1
                group by user_id, os,age,gender,yesterday)
    , r as (select reciever_id
                     , toDate(time) as yesterday
                     , count(user_id) as messages_received
                     , count(distinct user_id) as users_received
                from simulator_20220520.message_actions 
                where toDate(time) =today()-1
                group by reciever_id,yesterday)


    select if(user_id = 0,reciever_id, user_id) as user_id
    ,if(os='','no info',os) as os
    ,age
    ,multiIf(age = 0, 'no info',
         age <=20 , '1-20',
         age <=30 , '21-30',
         age <=40 , '31-40',
         age <=50 , '41-50',
         '50+') as age_type
    ,if(age=0,2,gender) as gender
    ,if(s.yesterday ='1970-01-01',r.yesterday,s.yesterday) as yesterday
    ,messages_sent
    ,users_sent
    ,messages_received
    ,users_received
    from s
    full join r on r.reciever_id = s.user_id
    '''
    ).df
    return message

def foo3(feed,message):
    df = feed.merge(message, left_on = ['user_id','os','age','age_type','gender','yesterday'], right_on =['user_id','os','age','age_type','gender','yesterday'], how ='outer').fillna(0)
    return df

def foo4(df):
    #TASK 4 - пол
    df_gender = df[['gender','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','gender'], as_index=False).agg('sum')
    df_gender['metric'] ='gender'
    df_gender = df_gender.rename(columns = {'gender':'metric_value'})
    df_gender['metric_value']= df_gender['metric_value'].apply(lambda x: 'no info' if x == 2 else x)
    return df_gender
    
def foo5(df):
    #TASK 5 - возраст
    df_age = df[['age_type','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','age_type'], as_index=False).agg('sum')
    df_age['metric'] ='age_type'
    df_age = df_age.rename(columns = {'age_type':'metric_value'})
    return df_age

def foo6(df):
    #TASK 6 - os
    df_os = df[['os','yesterday','likes','views','messages_sent','users_sent','messages_received','users_received']].groupby(['yesterday','os'], as_index=False).agg('sum')
    df_os['metric'] ='os'
    df_os = df_os.rename(columns = {'os':'metric_value'})
    return df_os

def foo7(df_gender,df_os,df_age):
    #TASK 7
    df_final = pd.concat([df_gender,df_os,df_age])
    df_final = df_final[['yesterday','metric','metric_value','likes','views','messages_sent','users_sent','messages_received','users_received']].rename(columns={'yesterday':'eventdate'})
    df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']] = df_final[['likes','views','messages_sent','users_sent','messages_received','users_received']].astype(int)
    return df_final

def into_base(df_final):
    #подключение к новой базе
    connection_ch = {
        'host': 'https://clickhouse.lab.karpov.courses',
        'password': '656e2b0c9c',
        'user': 'student-rw',
        'database': 'test'}

    q1 = '''
    CREATE TABLE IF NOT EXISTS test.eorlova
    (eventdate Date
    , metric String
    , metric_value String
    , likes UInt64
    , views UInt64
    , messages_sent UInt64
    , users_sent UInt64
    , messages_received UInt64
    , users_received UInt64) 
    ENGINE = MergeTree()
    ORDER BY eventdate
    '''
    pandahouse.execute(query=q1, connection=connection_ch)
    
    q2 = 'SELECT max(eventdate) as eventdate FROM test.eorlova'
    max_date = pandahouse.read_clickhouse(q2, connection=connection_ch)
    max_date = max_date.iloc[0]

    if (max_date == df_final.eventdate.max())[0]:
        q3 = '''
        ALTER TABLE test.eorlova DELETE WHERE eventdate == today()-1
        '''
        pandahouse.execute(query=q3, connection=connection_ch)
        pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
    else:
        pandahouse.to_clickhouse(df_final, table='eorlova',index=False, connection=connection_ch)
     
    return 

In [84]:
connection_ch = {
        'host': 'https://clickhouse.lab.karpov.courses',
        'password': '656e2b0c9c',
        'user': 'student-rw',
        'database': 'test'}

q1 = '''
alter table test.eorlova delete where eventdate = '2022.06.09'
    '''
pandahouse.execute(query=q1, connection=connection_ch)

b''